In [113]:
import pandas 
import numpy

from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA

from sklearn.svm import SVR

from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import ensemble

%matplotlib inline

In [114]:
seed = 7
numpy.random.seed(seed)

In [115]:

fdata=pandas.read_csv("train.csv")


In [116]:
fdata.columns = [ "x_coord", "y_coord", "month", "day", "ffmc", "dmc", "dc", "isi", "temp",
              "rh", "wind", "rain", "area" ]



In [117]:
from sklearn.preprocessing import LabelEncoder

le_month = LabelEncoder()
fdata["month"] = le_month.fit_transform(fdata["month"])

le_day = LabelEncoder()
fdata["day"] = le_day.fit_transform(fdata["day"])
fdata.pop("rain")

0      0.0
1      0.0
2      0.2
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
      ... 
419    0.0
420    0.0
421    0.0
422    0.0
423    0.0
424    0.0
425    0.0
426    0.0
427    6.4
428    0.8
429    0.8
430    0.4
431    0.0
432    0.0
433    0.0
434    0.0
435    0.0
436    0.0
437    1.4
438    0.0
439    0.0
440    0.0
441    0.0
442    0.0
443    0.0
444    0.0
445    0.0
446    0.0
447    0.0
448    0.0
Name: rain, Length: 449, dtype: float64

In [118]:
y = fdata.pop("area")
fdata=fdata-fdata.mean()
fdata=fdata/fdata.std()
fdata.head()

,x_coord,y_coord,month,day,ffmc,dmc,dc,isi,temp,rh,wind
0,1.014859,0.565490,0.453343,-1.431210,-0.819008,-1.305249,-1.816799,-0.865302,-1.845231,0.392644,1.511376
1,1.014859,-0.245708,0.964136,1.147495,-0.018636,-1.165781,0.486415,-0.520945,-0.170855,-0.714983,-1.798906
2,1.443122,1.376689,0.453343,-1.431210,0.181457,-1.197616,-1.884116,-0.025932,-1.828145,3.223248,-0.029617
3,1.443122,1.376689,-1.079036,0.116013,0.290599,-0.409318,-0.239250,1.200839,0.546735,-0.961122,0.769416
4,1.443122,1.376689,-1.079036,-0.915469,0.290599,-0.354744,-0.208797,-0.133544,0.871359,-1.084192,-0.543282


In [119]:
# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(fdata)
# update dataframe 
df_fe = pandas.concat((fdata, pandas.DataFrame(ugriz)), axis=1)
df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
df_fe.drop([ "x_coord", "y_coord", "month", "day", "ffmc", "dmc", "dc", "isi", "temp",
              "rh", "wind"], axis=1, inplace=True)
df_fe.head()

,PCA_1,PCA_2,PCA_3
0,3.476229,0.881149,0.382137
1,0.131486,-0.362108,-1.203728
2,3.326372,2.155000,-0.635142
3,-0.609451,2.253027,1.095524
4,-0.388539,1.712462,-0.354935


In [120]:

y=(y+1)
    
y= numpy.log(y)


In [121]:





X=df_fe

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=40)

In [123]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [124]:
predictions = lr_model.predict(X_test)
#predictions=numpy.expm1(predictions)
#predictions=numpy.expm1(predictions)
    # Evaluate the model
rms = sqrt(mean_squared_error(y_test, predictions))
print(rms)

1.2432897928453404


In [125]:
from sklearn.model_selection import cross_val_score

lr_scores = cross_val_score( lr_model, X, y,
                         scoring='neg_mean_squared_error', cv=10)

lr_rmse_scores = numpy.sqrt(-lr_scores)
print(lr_rmse_scores.mean())

1.3468933395073706


In [126]:
#print(X.describe())
rng = numpy.random.RandomState(10)

In [165]:
from sklearn.grid_search import GridSearchCV
clf_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                     n_estimators=65, random_state=rng,learning_rate=0.01)
param_grid = {"n_estimators" : [1, 10, 50, 100, 200, 300, 400, 500, 600]}
grid_search = GridSearchCV(clf_1, param_grid=param_grid, cv=10)
grid_search.fit(X_train, y_train)
print('Best parameters of Adaboost ' , grid_search.best_params_)
print('Best scrore of Adaboost :', grid_search.best_score_)



/home/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Best parameters of Adaboost  {'n_estimators': 100}
Best scrore of Adaboost : -0.21305451371571446


In [162]:
predictions = clf_1.predict(X_test)
#predictions=numpy.expm1(predictions)
#predictions=numpy.expm1(predictions)
    # Evaluate the model
rms = sqrt(mean_squared_error(y_test, predictions))
print(rms)

1.3409912098808752


In [129]:
from sklearn.tree import DecisionTreeRegressor

clf_2 = DecisionTreeRegressor()
clf_2.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [130]:
predictions = clf_2.predict(X_test)
#predictions=numpy.expm1(predictions)
#predictions=numpy.expm1(predictions)
    # Evaluate the model
rms = sqrt(mean_squared_error(y_test, predictions))
print(rms)

1.8648806997471705


In [131]:
from sklearn.model_selection import cross_val_score

dtr_scores = cross_val_score(clf_2, X, y,
                         scoring='neg_mean_squared_error', cv=10)

dtr_rmse_scores = numpy.sqrt(-dtr_scores)
print(dtr_rmse_scores.mean())

1.8683350754657475


In [132]:
test=pandas.read_csv("test.csv")

In [133]:
test.columns = ["Id", "x_coord", "y_coord", "month", "day", "ffmc", "dmc", "dc", "isi", "temp",
              "rh", "wind","rain"]

In [134]:
from sklearn.preprocessing import LabelEncoder

le_month = LabelEncoder()
test["month"] = le_month.fit_transform(test["month"])

le_day = LabelEncoder()
test["day"] = le_day.fit_transform(test["day"])

test.pop("rain")

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
     ..
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
59    0
60    0
61    0
62    0
63    0
64    0
65    0
66    0
Name: rain, Length: 67, dtype: int64

In [135]:


test.pop("Id")
test=test-test.mean()
test=test/test.std()
test.head()

,x_coord,y_coord,month,day,ffmc,dmc,dc,isi,temp,rh,wind
0,0.985834,-0.209635,0.573502,-0.253046,0.070840,-1.191830,0.579906,-0.409444,-0.633635,-0.557615,-1.210584
1,1.450981,1.442779,-1.331852,-0.799953,0.228466,0.975102,0.253111,0.622832,-1.857187,2.610531,-0.770305
2,1.450981,1.442779,0.891061,1.387674,0.140896,0.636319,0.603575,-0.332023,-0.911715,1.235675,0.795130
3,0.520687,0.616572,0.891061,-1.346860,0.543717,0.885612,0.692022,1.448652,0.905074,0.099925,0.795130
4,0.985834,-0.209635,0.891061,-0.799953,0.123382,0.572398,0.578245,-0.332023,0.256221,0.339030,-1.210584


In [136]:
# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(test)
# update dataframe 
dft_fe = pandas.concat((test, pandas.DataFrame(ugriz)), axis=1)
dft_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
dft_fe.drop([ "x_coord", "y_coord", "month", "day", "ffmc", "dmc", "dc", "isi", "temp",
              "rh", "wind"], axis=1, inplace=True)
dft_fe.head()

,PCA_1,PCA_2,PCA_3
0,0.536180,0.385638,-1.179854
1,0.902645,1.801226,2.910436
2,0.476997,1.764179,0.010823
3,-1.739635,-0.194320,0.056221
4,-0.610931,0.531070,-0.452547


In [137]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [138]:
predictions = lr_model.predict(dft_fe)
predictions=numpy.expm1(predictions)


In [166]:

adr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6),
                     n_estimators=100, random_state=rng)

adr.fit(X, y)


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=6, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=100,
         random_state=<mtrand.RandomState object at 0x7f3aca5ebd38>)

In [167]:
predictions = adr.predict(dft_fe)
predictions=numpy.expm1(predictions)


In [168]:
#ans=pandas.read_csv("ans.csv")
ans=pandas.read_csv("myteam_adr.csv")
#sampleSubmission = .values
# print(ans)

#X =sampleSubmission[:,0:1]
raw_data={'area' : predictions}
#@ID={'Id':X}
#ans.insert(1,'area', predictions, allow_duplicates=True)
s = pandas.DataFrame(raw_data, columns = ['area'])
ans=pandas.concat([ans,s],axis=1)
#ans['area'] = predictions
ans.to_csv('myteam_adr.csv')